In [128]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

### Load the Data

In [129]:
#relative path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_path = os.path.join(parent_dir, 'Data', 'train.csv')
test_path = os.path.join(parent_dir, 'Data', 'test.csv')

# Load data and test data
data = pd.read_csv(data_path, index_col = 0)
test = pd.read_csv(test_path, index_col = 0)

# Separate data into X and Y
y = data.SalePrice
X = data.drop("SalePrice", axis = 1)

### Examine All Features

In [130]:
print("The shape of X is", X.shape)
print("The shape of test set is", test.shape)
print("The columns of X are:\n", X.columns)

The shape of X is (1460, 79)
The shape of test set is (1459, 79)
The columns of X are:
 Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageF

In [131]:
num_name = X.select_dtypes(include=[np.number]).columns
cat_name = X.select_dtypes(include=[np.object_]).columns
print(num_name.shape[0], "numerical variables")
print(cat_name.shape[0], "categorical variables")

36 numerical variables
43 categorical variables


### Missing Value

In [132]:
# Number of empty entries in each column
# Data set
col_missing = X.isnull().sum(axis = 0)
col_missing = col_missing[col_missing > 0]
print(col_missing)
print()

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64



In [133]:
# Test set
col_missing_test = test.isnull().sum(axis = 0)
col_missing_test = col_missing_test[col_missing_test > 0]
print(col_missing_test)
len(col_missing_test)

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType       894
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64


33

In [134]:
# All rows have missing entry in data set
row_with_missing = [row for index, row in X.iterrows() if row.isnull().any()]
len(row_with_missing)

1460

### Three Feature Sets

#### Set 1: Numerical Variables without missing entries

In [135]:
num_no_missing = num_name.difference(col_missing.index).difference(col_missing_test.index)
len(num_no_missing)

25

In [136]:
num_no_missing

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'EnclosedPorch',
       'Fireplaces', 'FullBath', 'GrLivArea', 'HalfBath', 'KitchenAbvGr',
       'LotArea', 'LowQualFinSF', 'MSSubClass', 'MiscVal', 'MoSold',
       'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'ScreenPorch',
       'TotRmsAbvGrd', 'WoodDeckSF', 'YearBuilt', 'YearRemodAdd', 'YrSold'],
      dtype='object')

#### Set 2: All Numerical Variables

In [137]:
len(num_name)

36

#### Set 3: Numerical and Categorical Variables without missing entries

In [138]:
col_no_missing = X.columns.difference(col_missing.index).difference(col_missing_test.index)
len(col_no_missing)

45

In [139]:
# Numerical variables are good to go
# Try One Hot Encoding and Ordinal Encoding For Categoricals
col_no_missing.difference(num_name)
print(len(col_no_missing.difference(num_name)))

20


In [140]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop="first", sparse_output=False) # Handle unknown by default is error

# One Hot Encoding Variables
ohe_var = ["BldgType", "CentralAir", "Condition1", "Condition2", "Foundation", 
                                             "Heating", "HouseStyle", "LandContour", "LotConfig", "Neighborhood", 
                                             "RoofMatl", "RoofStyle", "SaleCondition", "Street"]

# One Hot Encoding Matrix
ohe_matrix = pd.DataFrame(ohe.fit_transform(X[ohe_var]))
ohe_matrix_test = pd.DataFrame(ohe.transform(test[ohe_var]))

# Put index and column name back
ohe_matrix.index = X.index
ohe_matrix_test.index = test.index

ohe_columns = ohe.get_feature_names_out(ohe_var).astype(str)
ohe_matrix.columns = ohe_columns
ohe_matrix_test.columns = ohe_columns

# Concat back to "data" and test
data = pd.concat([data, ohe_matrix], axis = 1)

test = pd.concat([test, ohe_matrix_test], axis = 1)

In [141]:
data[list(col_no_missing.difference(ohe_var))]

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,EnclosedPorch,ExterCond,ExterQual,Fireplaces,FullBath,GrLivArea,...,OverallCond,OverallQual,PavedDrive,PoolArea,ScreenPorch,TotRmsAbvGrd,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,856,854,0,3,0,TA,Gd,0,2,1710,...,5,7,Y,0,0,8,0,2003,2003,2008
2,1262,0,0,3,0,TA,TA,1,2,1262,...,8,6,Y,0,0,6,298,1976,1976,2007
3,920,866,0,3,0,TA,Gd,1,2,1786,...,5,7,Y,0,0,6,0,2001,2002,2008
4,961,756,0,3,272,TA,TA,1,1,1717,...,5,7,Y,0,0,7,0,1915,1970,2006
5,1145,1053,0,4,0,TA,Gd,1,2,2198,...,5,8,Y,0,0,9,192,2000,2000,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,953,694,0,3,0,TA,TA,1,2,1647,...,5,6,Y,0,0,7,0,1999,2000,2007
1457,2073,0,0,3,0,TA,TA,2,2,2073,...,6,6,Y,0,0,7,349,1978,1988,2010
1458,1188,1152,0,4,0,Gd,Ex,2,2,2340,...,9,7,Y,0,0,9,0,1941,2006,2010


In [142]:
# Ordinal Encoding

ord_var = ["ExterCond","ExterQual","HeatingQC","LandSlope","LotShape","PavedDrive"]

#### Set 4: Final Data

In [143]:
# TBD: address some missing values, ordinal encoding etc. 

### Save Clean Data

In [144]:
clean_data_path = os.path.join(parent_dir, 'Data', 'train_clean.csv')
clean_test_path = os.path.join(parent_dir, 'Data', 'test_clean.csv')

clean_feat = list(col_no_missing.difference(ohe_var).difference(ord_var)) + list(ohe_columns) + [y.name]
clean_feat_test = list(col_no_missing.difference(ohe_var).difference(ord_var)) + list(ohe_columns)
data[clean_feat].to_csv(clean_data_path)
test[clean_feat_test].to_csv(clean_test_path)